In [1]:
import torch
print(torch.cuda.is_available() , torch.__version__)

False 2.2.0+cpu


In [1]:
# %% test a specific model
from src.interface import ModelTrainer
ModelTrainer.main(stage = 0 , resume = 0 , checkname= 1)

24-05-15 06:01:25|MOD:display     |: Model Specifics:
24-05-15 06:01:25|MOD:display     |: Start Process [Data] at Wed May 15 06:01:25 2024!


--Process Queue : Data + Fit + Test
--Model_name is set to gru_day_ShortTest!
Callback : DynamicDataLink() , assign / unlink dynamic data in tra networks
Callback : ResetOptimizer(num_reset=2,trigger=40,recover_level=1.0,speedup2x=True) , reset optimizer on some epoch (can speedup scheduler)
Callback : CallbackTimer(verbosity=10) , record time cost of callback hooks
Callback : EarlyStoppage(patience=20) , stop fitting when validation score cease to improve
Callback : ValidationConverge(patience=5,eps=1e-05) , stop fitting when valid_score converge
Callback : EarlyExitRetrain(earliest=20,max_attempt=4,lr_multiplier=[1, 0.1, 10, 0.01, 100, 1]) , retrain with new lr if fitting stopped too early
Callback : NanLossRetrain(max_attempt=4) , retrain if fitting encounters nan loss
Callback : BatchDisplay(verbosity=10) , display batch progress bar
Callback : StatusDisplay(verbosity=10) , display epoch / event information
{'random_seed': None,
 'model_name': 'gru_day_ShortTest',
 'model_module': 

24-05-15 06:01:27|MOD:display     |: Finish Process [Data], Cost 1.8 Secs
24-05-15 06:01:27|MOD:display     |: Start Process [Fit] at Wed May 15 06:01:27 2024!


Pre-Norming method of [day] : {'divlast': True, 'histnorm': True}


  0%|          | 0/10 [00:00<?, ?it/s]

...

tree
version=v4
num_class=1
num_tree_per_iteration=1
label_index=0
max_feature_idx=31
objective=regression
feature_names=Column_0 Column_1 Column_2 Column_3 Column_4 Column_5 Column_6 Column_7 Column_8 Column_9 Column_10 Column_11 Column_12 Column_13 Column_14 Column_15 Column_16 Column_17 Column_18 Column_19 Column_20 Column_21 Column_22 Column_23 Column_24 Column_25 Column_26 Column_27 Column_28 Column_29 Column_30 Column_31
monotone_constraints=1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
feature_infos=[-0.09704279899597168:-0.010467920452356339] [-0.15362736582756042:-0.035090312361717224] [0.073884956538677216:0.12630784511566162] [-0.46400195360183716:-0.09097038209438324] [0.11509542167186737:0.19383078813552856] [-0.12885263562202454:0.36082196235656738] [-0.086226716637611389:0.23044149577617645] [0.07159925252199173:0.22961580753326416] [0.0045385099947452545:0.17100979387760162] [-0.24594894051551819:0.032304886728525162] [-0.15050864219665527:-0.086340442

24-05-15 06:02:42|MOD:display     |: gru_day_ShortTest #0 @20170103|FirstBite Ep#  4 Max Epoch|Train 0.0519 Valid 0.0522 BestVal 0.0699|Cost  1.2Min, 14.7Sec/Ep
24-05-15 06:02:42|MOD:display     |: Finish Process [Fit], Cost 0.0 Hours, 1.3 Min/model, 18.9 Sec/Epoch
24-05-15 06:02:42|MOD:display     |: Start Process [Test] at Wed May 15 06:02:42 2024!
24-05-15 06:02:42|MOD:display     |: Each Model Date Testing Mean Score(spearman):
Models            0       0       0
Output         best swalast swabest


tree
version=v4
num_class=1
num_tree_per_iteration=1
label_index=0
max_feature_idx=31
objective=regression
feature_names=Column_0 Column_1 Column_2 Column_3 Column_4 Column_5 Column_6 Column_7 Column_8 Column_9 Column_10 Column_11 Column_12 Column_13 Column_14 Column_15 Column_16 Column_17 Column_18 Column_19 Column_20 Column_21 Column_22 Column_23 Column_24 Column_25 Column_26 Column_27 Column_28 Column_29 Column_30 Column_31
monotone_constraints=1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
feature_infos=[-0.09704279899597168:-0.010467920452356339] [-0.15362736582756042:-0.035090312361717224] [0.073884956538677216:0.12630784511566162] [-0.46400195360183716:-0.09097038209438324] [0.11509542167186737:0.19383078813552856] [-0.12885263562202454:0.36082196235656738] [-0.086226716637611389:0.23044149577617645] [0.07159925252199173:0.22961580753326416] [0.0045385099947452545:0.17100979387760162] [-0.24594894051551819:0.032304886728525162] [-0.15050864219665527:-0.086340442

Test swabest 20170228 score : 0.02121: 100%|██████████| 35/35 [00:05<00:00,  6.86it/s] 
20170103     0.0156  0.0384  0.0212
AllTimeAvg   0.0156  0.0384  0.0212
AllTimeSum     0.55    1.34    0.74
Std          0.0470  0.0258  0.0599
TValue         1.96    8.80    2.10
AnnIR        1.6227  7.2860  1.7353
24-05-15 06:02:58|MOD:display     |: Finish Process [Test], Cost 15.5 Secs
24-05-15 06:02:58|MOD:__init__    |: Main Process Finished! Cost 1 Minutes 32.8 Seconds


                    hook_name  num_calls  total_time      avg_time
8              on_train_batch         40   40.884831  1.022121e+00
17           on_fit_model_end          1   27.352937  2.735294e+01
23              on_test_batch        105   14.594675  1.389969e-01
13        on_validation_batch         40    5.130381  1.282595e-01
4          on_fit_model_start          1    1.904187  1.904187e+00
20        on_test_model_start          1    0.604700  6.047001e-01
24          on_test_batch_end        105    0.126797  1.207592e-03
9          on_train_batch_end         40    0.070028  1.750708e-03
16           on_fit_epoch_end          4    0.041076  1.026899e-02
21   on_test_model_type_start          3    0.036259  1.208623e-02
14    on_validation_batch_end         40    0.023578  5.894601e-04
27                on_test_end          1    0.016889  1.688933e-02
10         on_train_epoch_end          4    0.012953  3.238320e-03
11  on_validation_epoch_start          4    0.008060  2.014875

In [ ]:
from src.interface import ModelTrainer
ModelTrainer.main(stage = 0 , resume = 0 , checkname= 1)

In [ ]:
# prepare data
from src.interface import DataModule
DataModule.prepare_data()